#Prediction Challenge 4 – Deep NLP
Create a ‘poem’ in the style of William Blake.
From the Gutenberg Dataset, use the 'blake-poems.txt' corpus as the source text. Use word 
level text generation. Use RNN, LSTM or GRU layers in the model. Make each poem you generate 25 
words long. Generate 3 poems and calculate the BLEU score on each of them.

In [1]:
# import necessary libraries
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from random import randint
import re
import keras
import nltk 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
nltk.download('gutenberg')  # downloads a library that NLTK uses

from nltk.corpus import gutenberg as gut  # downloads the gutenberg dataset

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [3]:
# get the book text
book_text = nltk.corpus.gutenberg.raw('blake-poems.txt')

In [ ]:


print(book_text)

[Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS OF INNOCENCE
 
 
 INTRODUCTION
 
 Piping down the valleys wild,
   Piping songs of pleasant glee,
 On a cloud I saw a child,
   And he laughing said to me:
 
 "Pipe a song about a Lamb!"
   So I piped with merry cheer.
 "Piper, pipe that song again;"
   So I piped: he wept to hear.
 
 "Drop thy pipe, thy happy pipe;
   Sing thy songs of happy cheer:!"
 So I sang the same again,
   While he wept with joy to hear.
 
 "Piper, sit thee down and write
   In a book, that all may read."
 So he vanish'd from my sight;
   And I pluck'd a hollow reed,
 
 And I made a rural pen,
   And I stain'd the water clear,
 And I wrote my happy songs
   Every child may joy to hear.
 
 
 THE SHEPHERD
 
 How sweet is the Shepherd's sweet lot!
 From the morn to the evening he stays;
 He shall follow his sheep all the day,
 And his tongue shall be filled with praise.
 
 For he hears the lambs' innocent call,
 And

data Pre-processing

In [4]:
#Data preprocessing
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()
book_text = preprocess_text(book_text)


In [ ]:
print(book_text[:500])

 poems by william blake songs of innocence and of experience and the book of thel songs of innocence introduction piping down the valleys wild piping songs of pleasant glee on cloud saw child and he laughing said to me pipe song about lamb so piped with merry cheer piper pipe that song again so piped he wept to hear drop thy pipe thy happy pipe sing thy songs of happy cheer so sang the same again while he wept with joy to hear piper sit thee down and write in book that all may read so he vanish 


In [ ]:

book_text


' poems by william blake songs of innocence and of experience and the book of thel songs of innocence introduction piping down the valleys wild piping songs of pleasant glee on cloud saw child and he laughing said to me pipe song about lamb so piped with merry cheer piper pipe that song again so piped he wept to hear drop thy pipe thy happy pipe sing thy songs of happy cheer so sang the same again while he wept with joy to hear piper sit thee down and write in book that all may read so he vanish from my sight and pluck a hollow reed and made rural pen and stain the water clear and wrote my happy songs every child may joy to hear the shepherd how sweet is the shepherd sweet lot from the morn to the evening he stays he shall follow his sheep all the day and his tongue shall be filled with praise for he hears the lambs innocent call and he hears the ewes tender reply he is watching while they are in peace for they know when their shepherd is nigh the echoing green the sun does arise and m

In [5]:
# convert words to numbers

book_text_words = (word_tokenize(book_text))
n_words = len(book_text_words)
unique_words = len(set(book_text_words))

tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(book_text_words)

vocab_size = len(tokenizer.word_index) + 1    # word_index is the dictionary. Store the number of unique words in vocab_size variable
word_2_index = tokenizer.word_index           # store the dictionary in the variable called word_2_index

# Create the input sequences
input_sequence_words = []  # input sequences in words (used for metric evaluation later on)
input_sequence = []   # empty list to hold the sequences that will be input into our model
output_words = []     # empty list to hold the output words
input_seq_length = 25  # length of the input sequence
for i in range(0, n_words - input_seq_length , 1):
    in_seq = book_text_words[i:i + input_seq_length]
    input_sequence_words.append(in_seq)
    out_seq = book_text_words[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

# reshape the input sequences to be 3-dimensional
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))

# Normalise the data by dividing by the max number of unique words (the vocab size)
X = X / float(vocab_size)

# one-hot encode the output words so that they can be used by the model (converts the output to 2-dimensions)
y = to_categorical(output_words)

     

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (6565, 25, 1)
y shape: (6565, 1506)


#Model1

In [ ]:
model_1 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model_1.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_1.add(LSTM(128, return_sequences=True))
model_1.add(LSTM(64))
model_1.add(Dense(y.shape[1], activation='softmax'))

model_1.compile(loss='categorical_crossentropy', optimizer='adam')
print(model_1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 25, 128)           131584    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1506)              97890     
                                                                 
Total params: 345,442
Trainable params: 345,442
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model_1.fit(X, y, batch_size=50, epochs=100, verbose=1)


Epoch 1/100
132/132 [==============================] - 28s 150ms/step - loss: 6.5650
Epoch 2/100
132/132 [==============================] - 26s 201ms/step - loss: 6.2066
Epoch 3/100
132/132 [==============================] - 25s 191ms/step - loss: 6.1885
Epoch 4/100
132/132 [==============================] - 26s 196ms/step - loss: 6.1822
Epoch 5/100
132/132 [==============================] - 21s 159ms/step - loss: 6.1777
Epoch 6/100
132/132 [==============================] - 21s 161ms/step - loss: 6.1756
Epoch 7/100
132/132 [==============================] - 21s 156ms/step - loss: 6.1737
Epoch 8/100
132/132 [==============================] - 21s 160ms/step - loss: 6.1731
Epoch 9/100
132/132 [==============================] - 22s 164ms/step - loss: 6.1702
Epoch 10/100
132/132 [==============================] - 27s 204ms/step - loss: 6.1685
Epoch 11/100
132/132 [==============================] - 22s 164ms/step - loss: 6.1666
Epoch 12/100
132/132 [==============================] - 22s 167

In [ ]:
seq = [' '.join(w) for w in input_sequence_words]
reference = seq

def generate_poem(PoemLength):
      # Make Predictions
    random_seq_index = np.random.randint(0, PoemLength-1)               # select a random number from within the range of the number of input sequences
    random_seq = input_sequence[random_seq_index]                       # get the input sequence that occurs at the randomly selected index (this is a list of integers)

    index_2_word = dict(map(reversed, word_2_index.items()))            # convert the integer sequence to its words
    seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

    word_sequence = []
    for i in range(PoemLength):
        int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
        int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)
        
        predicted_word_index = model_1.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
        predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
        word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

        random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
        random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

    # BLEU score
    candidate = ' '.join(word_sequence) # make the list of words into a string
    score = sentence_bleu(reference, candidate)

    for count, value in enumerate(candidate.split()):
      if count == 5 or count == 10 or count ==15  or count == 20:
        print(f'{value.title()}' , end= ' ')
        print('\t')
      else: 
        print(f'{value.title()}' , end= ' ')
        
    print('\n')
    print('BLEU Score for predicted words: %s'%(score))
    print('\n')

In [ ]:
# Predicting the poems

print('Predicted:Poem:1')
generate_poem(25)
print('\n')

print('Predicted-Poem:2')
generate_poem(25)
print('\n')

print('Predicted-Poem-3')
generate_poem(25)
print('\n')

Predicted:Poem:1
The The The The Southern And 	
The Spoil And The Mothers 	
The The Mothers And The 	
Mothers And The Mothers And 	
The Mothers Of The 

BLEU Score for predicted words: 0.8227413218542311




Predicted-Poem:2
The Dews And The Spoil And 	
The Mothers The The Mothers 	
The The Mothers And The 	
Mothers Of The Morning Of 	
The Evening And The 

BLEU Score for predicted words: 0.8918958059914224




Predicted-Poem-3
Glee And Stroke Symmetry The The 	
Golden Of The Chorus And 	
Closed And The Midst Of 	
The Chaste Of The Dead 	
Of The The The 

BLEU Score for predicted words: 0.9759356641738816






#model2

In [12]:
from keras.layers import Reshape

model_2 = Sequential()
model_2.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_2.add(LSTM(128, return_sequences=True))
model_2.add(LSTM(64, return_sequences=True))
model_2.add(LSTM(64))
model_2.add(Dense(y.shape[1], activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 25, 128)           66560     
                                                                 
 lstm_6 (LSTM)               (None, 25, 128)           131584    
                                                                 
 lstm_7 (LSTM)               (None, 25, 64)            49408     
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1506)              97890     
                                                                 
Total params: 378,466
Trainable params: 378,466
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model_2.fit(X, y, batch_size=50, epochs=100, verbose=1)


Epoch 1/100
132/132 [==============================] - 23s 126ms/step - loss: 6.5685 - accuracy: 0.0632
Epoch 2/100
132/132 [==============================] - 17s 126ms/step - loss: 6.2008 - accuracy: 0.0666
Epoch 3/100
132/132 [==============================] - 16s 125ms/step - loss: 6.1842 - accuracy: 0.0647
Epoch 4/100
132/132 [==============================] - 16s 122ms/step - loss: 6.1797 - accuracy: 0.0666
Epoch 5/100
132/132 [==============================] - 16s 122ms/step - loss: 6.1753 - accuracy: 0.0666
Epoch 6/100
132/132 [==============================] - 18s 133ms/step - loss: 6.1733 - accuracy: 0.0666
Epoch 7/100
132/132 [==============================] - 16s 122ms/step - loss: 6.1710 - accuracy: 0.0666
Epoch 8/100
132/132 [==============================] - 16s 124ms/step - loss: 6.1702 - accuracy: 0.0666
Epoch 9/100
132/132 [==============================] - 18s 132ms/step - loss: 6.1659 - accuracy: 0.0666
Epoch 10/100
132/132 [==============================] - 16s 122m

In [15]:
seq = [' '.join(w) for w in input_sequence_words]
reference = seq

def generate2_poem(PoemLength):
      # Make Predictions
    random_seq_index = np.random.randint(0, PoemLength-1)               # select a random number from within the range of the number of input sequences
    random_seq = input_sequence[random_seq_index]                       # get the input sequence that occurs at the randomly selected index (this is a list of integers)

    index_2_word = dict(map(reversed, word_2_index.items()))            # convert the integer sequence to its words
    seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

    word_sequence = []
    for i in range(PoemLength):
        int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
        int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)
        
        predicted_word_index = model_2.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
        predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
        word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

        random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
        random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

    # BLEU score
    candidate = ' '.join(word_sequence) # make the list of words into a string
    score = sentence_bleu(reference, candidate)

    for count, value in enumerate(candidate.split()):
      if count == 5 or count == 10 or count ==15  or count == 20:
        print(f'{value.title()}' , end= ' ')
        print('\t')
      else: 
        print(f'{value.title()}' , end= ' ')
        
    print('\n')
    print('BLEU Score for predicted words: %s'%(score))
    print('\n')

In [16]:
# Predicting the poems

print('Poem 1')
generate2_poem(25)
print('\n')

print('Poem 2')
generate2_poem(25)
print('\n')

print('Poem 3')
generate2_poem(25)
print('\n')

Poem 1
The The The The The The 	
The The The The The 	
The The The The The 	
The The The The The 	
The The The The 

BLEU Score for predicted words: 0.3990021400109723




Poem 2
And The The The The The 	
The The The The The 	
The The The The The 	
The The The The The 	
The The The The 

BLEU Score for predicted words: 0.43289874404366335




Poem 3
And The The The The The 	
The The The The The 	
The The The The The 	
The The The The The 	
The The The The 

BLEU Score for predicted words: 0.43289874404366335






#model3

In [18]:
model_3 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model_3.add(LSTM(150, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_3.add(LSTM(140, return_sequences=True))
model_3.add(LSTM(128))
model_3.add(Dense(y.shape[1], activation='softmax'))

model_3.summary()

model_3.compile(loss='categorical_crossentropy', optimizer='adam')
#print(model2.summary())
model_3.fit(X, y, batch_size=50, epochs=50, verbose=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 25, 150)           91200     
                                                                 
 lstm_13 (LSTM)              (None, 25, 140)           162960    
                                                                 
 lstm_14 (LSTM)              (None, 128)               137728    
                                                                 
 dense_4 (Dense)             (None, 1506)              194274    
                                                                 
Total params: 586,162
Trainable params: 586,162
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
132/132 [==============================] - 25s 149ms/step - loss: 6.5373
Epoch 2/50
132/132 [==============================] - 20s 151ms/step - loss: 6.2083
Epoch 3/50
1

In [19]:
seq = [' '.join(w) for w in input_sequence_words]
reference = seq

def generate3_poem(PoemLength):
      # Make Predictions
    random_seq_index = np.random.randint(0, PoemLength-1)               # select a random number from within the range of the number of input sequences
    random_seq = input_sequence[random_seq_index]                       # get the input sequence that occurs at the randomly selected index (this is a list of integers)

    index_2_word = dict(map(reversed, word_2_index.items()))            # convert the integer sequence to its words
    seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

    word_sequence = []
    for i in range(PoemLength):
        int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
        int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)
        
        predicted_word_index = model_3.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
        predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
        word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

        random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
        random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

    # BLEU score
    candidate = ' '.join(word_sequence) # make the list of words into a string
    score = sentence_bleu(reference, candidate)

    for count, value in enumerate(candidate.split()):
      if count == 5 or count == 10 or count ==15  or count == 20:
        print(f'{value.title()}' , end= ' ')
        print('\t')
      else: 
        print(f'{value.title()}' , end= ' ')
        
    print('\n')
    print('BLEU Score for predicted words: %s'%(score))
    print('\n')

In [23]:
for i in range(3):
    print(f'Poem {i+1}:')
    generate3_poem(PoemLength=25) # set the desired length of the poem (in number of words)


Poem 1:
The Mothers Of The Mothers Of 	
The Mothers Of The Mothers 	
Of The Mothers Of The 	
Mothers Of The Mothers Of 	
The Mothers Of The 

BLEU Score for predicted words: 0.6210484636312422


Poem 2:
And The The The The The 	
The The The The The 	
The The The The The 	
The The The The The 	
The The The The 

BLEU Score for predicted words: 0.43289874404366335


Poem 3:
The The The The The The 	
The The The The The 	
The The The The The 	
The The The The The 	
The The The The 

BLEU Score for predicted words: 0.3990021400109723




#model

In [9]:

# create, compile and fit the model
model = Sequential()
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=64, epochs=100, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 800)               2566400   
                                                                 
 dropout (Dropout)           (None, 800)               0         
                                                                 
 dense_1 (Dense)             (None, 1506)              1206306   
                                                                 
Total params: 3,772,706
Trainable params: 3,772,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
103/103 [==============================] - 55s 514ms/step - loss: 6.4925 - accuracy: 0.0599
Epoch 2/100
103/103 [==============================] - 54s 524ms/step - loss: 6.2188 - accuracy: 0.0647
Epoch 3/100
103/103 [==============================] - 52s 508ms/step - loss: 6.2001 - accuracy: 0.0

In [10]:
seq = [' '.join(w) for w in input_sequence_words]
reference = seq

def generate_poem(PoemLength):
      # Make Predictions
    random_seq_index = np.random.randint(0, PoemLength-1)               # select a random number from within the range of the number of input sequences
    random_seq = input_sequence[random_seq_index]                       # get the input sequence that occurs at the randomly selected index (this is a list of integers)

    index_2_word = dict(map(reversed, word_2_index.items()))            # convert the integer sequence to its words
    seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

    word_sequence = []
    for i in range(PoemLength):
        int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
        int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)
        
        predicted_word_index = model.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
        predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
        word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

        random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
        random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

    # BLEU score
    candidate = ' '.join(word_sequence) # make the list of words into a string
    score = sentence_bleu(reference, candidate)

    for count, value in enumerate(candidate.split()):
      if count == 5 or count == 10 or count ==15  or count == 20:
        print(f'{value.title()}' , end= ' ')
        print('\t')
      else: 
        print(f'{value.title()}' , end= ' ')
        
    print('\n')
    print('BLEU Score for predicted words: %s'%(score))
    print('\n')


In [11]:
# Predicting the poems

print('Poem 1')
generate_poem(25)
print('\n')

print('Poem 2')
generate_poem(25)
print('\n')

print('Poem 3')
generate_poem(25)
print('\n')

Poem 1
Glee On Cloud Saw Child And 	
He Laughing Said To Me 	
Pipe Song About Lamb So 	
Piped With Merry Cheer Piper 	
Pipe That Song Again 

BLEU Score for predicted words: 1.0




Poem 2
Song About Lamb So Piped With 	
Merry Cheer Piper Pipe That 	
Song Again So Piped He 	
Wept To Hear Drop Thy 	
Pipe Thy Happy Pipe 

BLEU Score for predicted words: 1.0




Poem 3
And He Laughing Said To Me 	
Pipe Song About Lamb So 	
Piped With Merry Cheer Piper 	
Pipe That Song Again So 	
Piped He Wept To 

BLEU Score for predicted words: 1.0






model 4

In [ ]:
model_4 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model_4.add(LSTM(150, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model_4.add(LSTM(140, return_sequences=True))
model_4.add(LSTM(128))
model_4.add(Dense(y.shape[1], activation='softmax'))

model_4.summary()



model_4.compile(loss='categorical_crossentropy', optimizer='adam')
#print(model2.summary())
model_4.fit(X, y, batch_size=50, epochs=50, verbose=1)

In [ ]:
seq = [' '.join(w) for w in input_sequence_words]
reference = seq

def generate4_poem(PoemLength):
      # Make Predictions
    random_seq_index = np.random.randint(0, PoemLength-1)               # select a random number from within the range of the number of input sequences
    random_seq = input_sequence[random_seq_index]                       # get the input sequence that occurs at the randomly selected index (this is a list of integers)

    index_2_word = dict(map(reversed, word_2_index.items()))            # convert the integer sequence to its words
    seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

    word_sequence = []
    for i in range(PoemLength):
        int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
        int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)
        
        predicted_word_index = model.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
        predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
        word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

        random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
        random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

    # BLEU score
    candidate = ' '.join(word_sequence) # make the list of words into a string
    score = sentence_bleu(reference, candidate)

    for count, value in enumerate(candidate.split()):
      if count == 5 or count == 10 or count ==15  or count == 20:
        print(f'{value.title()}' , end= ' ')
        print('\t')
      else: 
        print(f'{value.title()}' , end= ' ')
        
    print('\n')
    print('BLEU Score for predicted words: %s'%(score))
    print('\n')

In [ ]:
# Predicting the poems

print('Poem 1')
generate4_poem(25)
print('\n')

print('Poem 2')
generate4_poem(25)
print('\n')

print('Poem 3')
generate4_poem(25)
print('\n')

#conclusion:
the model with memory size 800 gives better result compare to other models